## Libraries und Vairablen

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

In [30]:
buchhaus_new_last_30 = 'https://www.buchhaus.ch/de/heute/last30'

## Webscraping

In [31]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
#load the page
driver.get(buchhaus_new_last_30)

# data frame
buch_df = pd.DataFrame(columns=['title',  
                            'preis',
                            'autor', 
                            'details', 
                            'genre',
                            'text'])

''' 
Algorith explanation:
1. go to the i'th book and click on it
2. extract all the necessary data
3. add the data to the data frame
4. go back to main page
5. after 25 books go to the next page
'''

for page in range(1, 10):
    for book in range(1,8):
        try:
            time.sleep(1)
            # go to the i'th book and click on it
            buch = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[4]/div/div/div[{}]/div/div[1]/div/div/span/span/a'.format(book))
            buch.click()

            # get the data simple data
            try: titel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/h1/span/span').text
            except: titel = 'nan'

            try: preis = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[2]/div/div/span[2]/span').text
            except: preis = 'nan'

            try: autor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div[2]/div').text
            except: autor = 'nan'

            # get the 'Buchbeschreibungen'
            try: text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[1]/div[2]/div/span/span/span').text
            except: text = 'nan'

            # get the 'Buchdetails'
            try: details = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[2]/div[2]/div').text
            except: details = 'nan'

        # Fehlermeldung
        except: 
            print('error mit Buch {} auf Seite {}'.format(book, page))
            continue
    

        df_temp = pd.DataFrame({'title': titel,  
                        'preis': preis,
                        'autor': autor,
                        'details': details,
                        'text': text
                        }, index=[0])
        buch_df = pd.concat([buch_df, df_temp], ignore_index=True)
    
        #back to the main page
        driver.get(buchhaus_new_last_30)

    # go to the next page
    if page == 1: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a').click()
    else: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]').click()
    time.sleep(1)


C:\Users\yanni\AppData\Local\Temp\ipykernel_15848\2562079825.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


error mit Buch 5 auf Seite 1
error mit Buch 6 auf Seite 1
error mit Buch 7 auf Seite 1
error mit Buch 5 auf Seite 2
error mit Buch 6 auf Seite 2
error mit Buch 7 auf Seite 2
error mit Buch 5 auf Seite 3
error mit Buch 6 auf Seite 3
error mit Buch 7 auf Seite 3
error mit Buch 5 auf Seite 4
error mit Buch 6 auf Seite 4
error mit Buch 7 auf Seite 4
error mit Buch 5 auf Seite 5
error mit Buch 6 auf Seite 5
error mit Buch 7 auf Seite 5
error mit Buch 5 auf Seite 6
error mit Buch 6 auf Seite 6
error mit Buch 7 auf Seite 6
error mit Buch 5 auf Seite 7
error mit Buch 6 auf Seite 7
error mit Buch 7 auf Seite 7
error mit Buch 5 auf Seite 8
error mit Buch 6 auf Seite 8
error mit Buch 7 auf Seite 8
error mit Buch 5 auf Seite 9
error mit Buch 6 auf Seite 9
error mit Buch 7 auf Seite 9


## Datawrangling

In [51]:
buch_df.shape

(36, 8)

In [38]:
buch_df['details'][0]

'ISBN/GTIN978-3-7645-0771-8ProduktartBuchEinbandartPaperbackVerlagBlanvaletErscheinungsdatum26.04.2023Auflage1. A.ReiheMilla Nova ermitteltReihen-Nr.05Seiten608 SeitenSpracheDeutsch'

In [46]:
'''
short algorithm description:
1. List all possible keys: These are the categories that precede the actual values in the data string.
2. Create a regular expression: This regular expression is designed to match each key, followed by any characters until the next key is found or until the end of the string.
3. Find all matches in the data: Use the 're.findall' function to apply the regular expression to the data string. 
This returns a list of tuples containing the key and the associated value for each match.
4. Convert the matches into a Dictionnary in the Dataframe.
'''

def extract_details(details):
    # Key words which can be find in the string
    keys = ["ISBN/GTIN", "Produktart", "Einbandart", "Verlag", "Erscheinungsdatum", "Auflage", "Reihe", "Reihen-Nr.", "Seiten", "Sprache", "Masse", "Artikel-Nr."]

    # generate a regex which can find all the keys in the string
    regex = "(" + "|".join(keys) + ")(.*?)(?=" + "|".join(keys) + "|$)"

    # finds all key value pairs in the string
    matches = re.findall(regex, details)
    
    # returns a dictionary with key value pairs
    return dict((k.strip(), v.strip()) for k, v in matches)

In [47]:
# apply the function to the data frame (create a new column with extracted details)
buch_df['details_dict'] = buch_df['details'].apply(extract_details)

In [49]:
# create new dataframe with the details
details_df = buch_df['details_dict'].apply(pd.Series)

In [53]:
# concat both dataframes
buch_df = pd.concat([buch_df, details_df], axis=1)

In [58]:
buch_df.drop(['details', 'details_dict'], axis=1, inplace=True)

In [59]:
buch_df

,title,preis,untertitel,autor,genre,text,ISBN/GTIN,Produktart,Einbandart,Verlag,Erscheinungsdatum,Auflage,Reihe,Seiten,Sprache,Masse,Artikel-Nr.
0,Der Feind,24.90,"Brand, Christine","Brand, Christine",NaN,Ein bizarre Mordserie an Männern sowie Schüsse...,978-3-7645-0771-8,Buch,Paperback,Blanvalet,26.04.2023,1. A.,n-Nr.05,,Deutsch,NaN,NaN
1,Die Krume Brot,29.00,nan,nan,NaN,"Adelina, Tochter italienischer Einwanderer, ar...",978-3-498-00320-3,Buch,Gebunden,Rowohlt,18.04.2023,,NaN,,Deutsch,"Breite 133 mm, Höhe 209 mm, Dicke 23 mmGewicht...",NaN
2,Der Magier im Kreml,36.90,"Da Empoli, GiulianoMeßner, MichaelaÜbersetzung","Da Empoli, Giuliano",NaN,SPIEGEL-BESTSELLER UND VIELFACH AUSGEZEICHNET\...,978-3-406-79993-8,Buch,Gebunden,Beck,24.04.2023,NaN,NaN,,Deutsch,"Breite 139 mm, Höhe 217 mm, Dicke 25 mmGewicht...",55652787
3,Elternabend,23.90,"Fitzek, Sebastian","Fitzek, Sebastian",NaN,Stell dir vor ...\n... du musst eine halbe Ewi...,978-3-426-28413-1,Buch,Paperback,Droemer/Knaur,26.04.2023Erstverkaufstag26.04.2023,,NaN,,Deutsch,"Breite 135 mm, Höhe 210 mm, Dicke 25 mm",NaN
4,Wo wir uns trafen,32.90,"Lundberg, SofiaDörries, MaikeÜbersetzungSchöps...","Lundberg, Sofia",NaN,Nur wahre Freundschaft berührt dein Herz ...\n...,978-3-442-31645-8,Buch,Gebunden,Goldmann,26.04.2023,1. A.,NaN,,Deutsch,"Breite 148 mm, Höhe 221 mm, Dicke 37 mmGewicht...",NaN
5,Going Zero,34.00,nan,nan,NaN,Hat man als Einzelner überhaupt eine Chance ge...,978-3-257-07192-4,Buch,Gebunden,Diogenes,26.04.2023,1. A.,NaN,,Deutsch,"Breite 116 mm, Höhe 184 mm, Dicke 30 mmGewicht...",NaN
6,Sommernächte in Paris,16.90,"Swan, KarenWittich, GertrudÜbersetzung","Swan, Karen",NaN,Ein unvergesslicher Sommer in der Stadt der Li...,978-3-442-49396-8,Taschenbuch,Paperback,Goldmann,20.04.2023,Deutsche Erstausgabe,NaN,,Deutsch,"Breite 126 mm, Höhe 187 mm, Dicke 35 mmGewicht...",NaN
7,One of the Girls,24.90,"Clarke, LucyHofstetter, UrbanÜbersetzung","Clarke, Lucy",NaN,Sechs Frauen. Sechs Geheimnisse. Eine Leiche.\...,978-3-423-26359-7,Buch,Paperback,DTV,20.04.2023,,NaN,,Deutsch,"Breite 136 mm, Höhe 210 mm, Dicke 31 mmGewicht...",NaN
8,Liebe oder Eierlikör - Fast eine Romanze,23.90,"Heldt, Dora","Heldt, Dora",NaN,Früher war mehr Romantik ...\n\nErnst Mannsen ...,978-3-423-28337-3,Buch,Gebunden,DTV,20.04.2023,,NaN,,Deutsch,"Breite 119 mm, Höhe 175 mm, Dicke 24 mmGewicht...",NaN
9,Damals im Tessin,26.90,nan,nan,NaN,"Zwanzig Jahre sind vergangen, seit Malvaglia, ...",978-3-7152-5503-3,Buch,Paperback,Atlantis Zürich,20.04.2023,NaN,NaN,,Deutsch,"Breite 125 mm, Höhe 205 mm, Dicke 33 mmGewicht...",55771950


## Ausblick

nächste Schritte:

* anpassung der Unterachiedlichen Buchinformationen

Ausblick:

* Bücher in den nächsten 30 Tagen könnten auch gescrapet werden